In [1]:
import os, sys, time
import torch
from IPython.display import HTML, display

from main import main
from network import modeling

In [2]:
models = {
    'deeplabv3plus_mobilenet': {'batch_size': 16, 'val_batch_size': 16},
    'deeplabv3plus_xception': {'batch_size': 16, 'val_batch_size': 16},
    'deeplabv3plus_resnet101': {'batch_size': 16, 'val_batch_size': 16},
    'segmenter_vit_large': {'batch_size': 1, 'val_batch_size': 1},
    'mae_segmenter_vit_base': {'batch_size': 2, 'val_batch_size': 2},
    'mae_segmenter_vit_base_deconv': {'batch_size': 2, 'val_batch_size': 2},
    'mae_segmenter_vit_base_mask': {'batch_size': 1, 'val_batch_size': 1},
    'mae_segmenter_vit_huge': {'batch_size': 1, 'val_batch_size': 1}
}

In [3]:
def train(model, use_ckpt=False):
    config = models[model]
    ckpt = f"checkpoints/latest_{model}_cityscapes_os16.pth"
    command = f"python main.py --model {model} --dataset cityscapes --gpu_id 0 --total_epochs 100 --base_lr 0.1 --loss_type focal_loss --crop_size 768 --batch_size {config['batch_size']} --val_batch_size {config['val_batch_size']} --use_amp --output_stride 16 --data_root ./datasets/data/cityscapes"
    if use_ckpt and os.path.exists(ckpt):
        command += f" --ckpt {ckpt} --continue_training"
    sys.argv = command.split()[1:]
    print("--------------------------------------------------")
    print(f"Train model {model} with command:")
    print(command)
    print("--------------------------------------------------")
    main()
    
def test(model):
    config = models[model]
    ckpt = f"checkpoints/best_{model}_cityscapes_os16.pth"
    if not os.path.exists(ckpt):
        print(f"Test model failed, cannot find trained weights {ckpt}")
    else:
        command = f"python main.py --model {model} --dataset cityscapes --gpu_id 0 --val_batch_size {config['val_batch_size']} --use_amp --output_stride 16 --data_root ./datasets/data/cityscapes --test_only --ckpt {ckpt}"
        sys.argv = command.split()[1:]
        print("--------------------------------------------------")
        print(f"Evaluate model {model} with command:")
        print(command)
        print("--------------------------------------------------")
        main()
        performance(model, ckpt)
        
def performance(model, ckpt):
    if not torch.cuda.is_available():
        return
    torch.cuda.empty_cache()
    device = torch.device('cuda')
    m = modeling.__dict__[model](num_classes=19, output_stride=16)
    m.load_state_dict(torch.load(ckpt, map_location=torch.device('cpu'))['model_state'])
    m = m.to(device)
    m.eval()
    with torch.no_grad():
        avg_time_cost = 0.0
        for i in range(10):
            X = torch.rand((1,3,1024,2048)).to(device)
            time_cost = time.time()
            Y = m(X)
            avg_time_cost += time.time() - time_cost
    del m
    torch.cuda.empty_cache()
    avg_time_cost /= 10.0
    print(f"Time cost per input: {avg_time_cost*1000:0.1f}ms, FPS={int(1/avg_time_cost):d}")

In [4]:
user_selection = {
    'model': 'deeplabv3plus_mobilenet',
    'use_ckpt': 'true',
    'action': 'train'
}

dropdown_options = ""
for model in models.keys():
    dropdown_options += f'<option value="{model}">{model}</option>'

dropdown_menu = f'''
<div style="margin-bottom: 10px;">
  <label for="model">Select Model:</label>
  <select id="model">
    {dropdown_options}
  </select>
</div>
<div style="margin-bottom: 10px;">
  <label for="ckpt">Use Checkpoint:</label>
  <select id="ckpt">
    <option value="true">True</option>
    <option value="false">False</option>
  </select>
</div>
<div style="margin-bottom: 10px;">
  <label for="action">Select Action:</label>
  <select id="action">
    <option value="train">Train</option>
    <option value="test">Test</option>
    <option value="train_test">Train and Test</option>
  </select>
</div>
'''

dropdown_menu += '''
<script>
  document.getElementById('model').onchange = function() {
    IPython.notebook.kernel.execute('user_selection["model"] = "' + this.value + '"');
  };
  document.getElementById('ckpt').onchange = function() {
    IPython.notebook.kernel.execute('user_selection["use_ckpt"] = "' + this.value + '"');
  };
  document.getElementById('action').onchange = function() {
    let ckpt = document.getElementById('ckpt')
    if (this.value === 'test') {
      ckpt.value = 'false'
      ckpt.disabled = true
    } else {
      ckpt.disabled = false
    }
    IPython.notebook.kernel.execute('user_selection["action"] = "' + this.value + '"');
  };
</script>
'''

display(HTML(dropdown_menu))

In [ ]:
if "train" in user_selection["action"]:
    train(user_selection["model"], user_selection["use_ckpt"]=='true')
if "test" in user_selection["action"]:
    test(user_selection["model"])

--------------------------------------------------
Train model mae_segmenter_vit_base_mask with command:
python main.py --model mae_segmenter_vit_base_mask --dataset cityscapes --gpu_id 0 --total_epochs 100 --base_lr 0.1 --loss_type focal_loss --crop_size 768 --batch_size 1 --val_batch_size 1 --use_amp --output_stride 16 --data_root ./datasets/data/cityscapes
--------------------------------------------------
Device: cuda
Dataset: cityscapes, Train set: 2975, Val set: 500
[!] Retrain


C:\Users\25816\anaconda3\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 1, Itrs 297/297500, Loss=1.193874
Epoch 1, Itrs 594/297500, Loss=0.671856
Epoch 1, Itrs 891/297500, Loss=0.528846
Epoch 1, Itrs 1188/297500, Loss=0.417039
Epoch 1, Itrs 1485/297500, Loss=0.462493
Epoch 1, Itrs 1782/297500, Loss=0.370647
Epoch 1, Itrs 2079/297500, Loss=0.383562
Epoch 1, Itrs 2376/297500, Loss=0.363222
Epoch 1, Itrs 2673/297500, Loss=0.317141
Epoch 1, Itrs 2970/297500, Loss=0.344087
Model saved as checkpoints/latest_mae_segmenter_vit_base_mask_cityscapes_os16.pth
validation...


500it [02:22,  3.52it/s]



Overall Acc: 0.865110
Mean Acc: 0.489327
FreqW Acc: 0.795907
Mean IoU: 0.364638

Model saved as checkpoints/best_mae_segmenter_vit_base_mask_cityscapes_os16.pth
Epoch 2, Itrs 3267/297500, Loss=0.287880
Epoch 2, Itrs 3564/297500, Loss=0.249993
Epoch 2, Itrs 3861/297500, Loss=0.246915
Epoch 2, Itrs 4158/297500, Loss=0.240841
Epoch 2, Itrs 4455/297500, Loss=0.250817
Epoch 2, Itrs 4752/297500, Loss=0.270381
Epoch 2, Itrs 5049/297500, Loss=0.235994
Epoch 2, Itrs 5346/297500, Loss=0.257131
Epoch 2, Itrs 5643/297500, Loss=0.221971
Epoch 2, Itrs 5940/297500, Loss=0.206606
Model saved as checkpoints/latest_mae_segmenter_vit_base_mask_cityscapes_os16.pth
validation...


...

500it [02:08,  3.88it/s]



Overall Acc: 0.956559
Mean Acc: 0.813995
FreqW Acc: 0.919507
Mean IoU: 0.746596

Epoch 94, Itrs 276804/297500, Loss=0.090264
Epoch 94, Itrs 277101/297500, Loss=0.093334
Epoch 94, Itrs 277398/297500, Loss=0.092052
Epoch 94, Itrs 277695/297500, Loss=0.093087
Epoch 94, Itrs 277992/297500, Loss=0.094674
Epoch 94, Itrs 278289/297500, Loss=0.091533
Epoch 94, Itrs 278586/297500, Loss=0.093622
Epoch 94, Itrs 278883/297500, Loss=0.089487
Epoch 94, Itrs 279180/297500, Loss=0.097873
Epoch 94, Itrs 279477/297500, Loss=0.089803
Model saved as checkpoints/latest_mae_segmenter_vit_base_mask_cityscapes_os16.pth
validation...


500it [02:05,  3.98it/s]



Overall Acc: 0.956649
Mean Acc: 0.823011
FreqW Acc: 0.919686
Mean IoU: 0.752443

Epoch 95, Itrs 279774/297500, Loss=0.091460
Epoch 95, Itrs 280071/297500, Loss=0.090337
Epoch 95, Itrs 280368/297500, Loss=0.093664
Epoch 95, Itrs 280665/297500, Loss=0.090864
Epoch 95, Itrs 280962/297500, Loss=0.090943
Epoch 95, Itrs 281259/297500, Loss=0.095891
Epoch 95, Itrs 281556/297500, Loss=0.090295
Epoch 95, Itrs 281853/297500, Loss=0.094954
Epoch 95, Itrs 282150/297500, Loss=0.092915
Epoch 95, Itrs 282447/297500, Loss=0.095467
Model saved as checkpoints/latest_mae_segmenter_vit_base_mask_cityscapes_os16.pth
validation...


49it [00:16,  2.92it/s]